# Project 3: 'AskFeminists' vs. 'MensRights'
## Part C: Vectorizing & Analysis

[**Vectorizing**](#vector)

Before combining the two subreddit dataframes into one for modeling, I tested out different vectorizing options on both to compare the two subreddits. I tried out CountVectorizer and TFIDF on the AskFeminists content setting ngrams = (3-5) to get the top phrases from both: TFIDF was less repetitive than the CountVectorizer so I used TFIDF for vectorizing throughout. 

I tried vectorizing on ngrams 3-5, 1-2, and just single words, and created two custom lists of stopwords: one with all English stopwords and words in common between the top 100 words (without stopwords) from each subreddit; the second list of stopwords was created after fitting TFIDF on both subreddits (with stopwords = the first custom list) and taking the common words from the top 100 lists of words for each subreddit again. I tested models with both sets of stopwords and ultimately they didn't improve the model over using stopwords = 'english.')

[**Modeling**](#model)

I primarily tested two models on the lemmatized text: Logistic Regression and Random Forest Classifier. Starting with Logistic Regression, I tried several different parameters for vectorizing with TFIDF and the best parameters were 125,000 max features, ngrams = 1-4, and stopwords = 'English'.

The best test score I had ended up being on Logistic Regression (76% accuracy compared to baseline of 50%). This model was overfit on the training data (85% accuracy), and setting lower max_features closed the gap to a 3% difference between training/testing scores but also lowered the test scores. 

The best training score I had was with RandomForestClassifier, n_estimators = 100 and max_depth = 880, which gave me 98% on training data (but just 72% on the test data.)

In [1]:
# Import libaries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score

import regex as re
import nltk
from nltk.corpus import stopwords # Import the stop word list

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import text

from sklearn.linear_model import LogisticRegression

pd.set_option('display.max_colwidth', -1)
pd.options.display.max_columns = 999

In [2]:
men = pd.read_csv('./data/men_clean_lem')
fem = pd.read_csv('./data/fem_clean_lem')

### Balancing datasets, removing nulls

In [3]:
men['lems'].isnull().sum()

225

In [4]:
men.dropna(inplace = True)

In [5]:
fem['lems'].isnull().sum()

350

In [6]:
fem.dropna(inplace = True)

In [7]:
men.shape

(31221, 8)

In [8]:
fem.shape

(28955, 8)

In [9]:
# creating smaller version for mensrights, same # rows as submissions for askfeminists
men = men.sample(29000, replace = False, random_state=42)

In [10]:
men.shape

(29000, 8)

## Vectorizing: Separate Analysis <a name="vector"></a>

In [11]:
# Instantiate a CountVectorizer
vect = CountVectorizer(ngram_range=(3,5), max_features = 10000, stop_words = 'english')
# Instantiate TFIDF
tfidf = TfidfVectorizer(ngram_range = (3, 5), max_features = 10000, stop_words = 'english')

In [12]:
# vectorizing fem for test
fem_vect = vect.fit_transform(fem['lems'])
# tfidfing fem for test
fem_tfidf = tfidf.fit_transform(fem['lems'])

In [13]:
# creating a df for vectorized words
fem_vect_df = pd.DataFrame(fem_vect.toarray(), columns=vect.get_feature_names())
# creating a df for tfidf words
fem_tfidf_df = pd.DataFrame(fem_tfidf.toarray(), columns=tfidf.get_feature_names())

In [14]:
# looking at vectorized value counts
vect_counts = fem_vect_df.sum(axis=0)
vect_counts.sort_values(ascending=False)[0:10]

reflect feminist perspective                  160
feminist reflect feminist                     155
feminist reflect feminist perspective         153
come feminist reflect                         89 
come feminist reflect feminist                89 
come feminist reflect feminist perspective    89 
level comment thread                          81 
difference men woman                          80 
feminist perspective comment                  74 
doesn make sense                              74 
dtype: int64

In [15]:
# looking at tfidf value counts
tfidf_counts = fem_tfidf_df.sum(axis=0)
tfidf_counts.sort_values(ascending=False)[0:10]

doesn make sense                41.643637
false rape accusation           36.430295
difference men woman            32.162463
traditional gender role         28.743795
just don think                  27.087392
reflect feminist perspective    26.147715
don really know                 25.665504
innocent proven guilty          24.234714
rape sexual assault             24.215631
gt don think                    23.785815
dtype: float64

TFIDF seems to have done a better job of ignoring similar phrases than count vectorizer (ex. 'reflect feminist perspective') and the results are more interesting.

##### Comparing Top Phrases (TFIDF, ngrams 3-5)

In [16]:
# fitting to tfidf
men_tfidf = tfidf.fit_transform(men['lems'])

In [17]:
# creating a df for tfidf words
men_tfidf_df = pd.DataFrame(men_tfidf.toarray(), columns=tfidf.get_feature_names())

In [18]:
# pulling top 10 phrases for men TFIDF with English stopwords, 3 are the same as fem
tfidf_counts_men = men_tfidf_df.sum(axis=0)
tfidf_counts_men.sort_values(ascending=False)[0:10]

false rape accusation     62.306164
men right movement        60.595215
men right issue           54.943756
international men day     50.388507
men right activist        39.744089
gender pay gap            33.950820
innocent proven guilty    33.152104
pay child support         27.741176
year old boy              22.977497
doesn make sense          22.798229
dtype: float64

#### Analyzing single words and bigrams

In [19]:
tfidf = TfidfVectorizer(ngram_range = (1, 2), max_features = 10000, stop_words = 'english')

In [20]:
# fitting to tfidf
fem_tfidf = tfidf.fit_transform(fem['lems'])

In [21]:
men_tfidf = tfidf.fit_transform(men['lems'])

In [22]:
# creating a df for tfidf words
fem_tfidf_df = pd.DataFrame(fem_tfidf.toarray(), columns=tfidf.get_feature_names())
men_tfidf_df = pd.DataFrame(men_tfidf.toarray(), columns=tfidf.get_feature_names())

In [23]:
# pulling top 10 words/phrases for fem TFIDF with English stopwords
tfidf_counts_fem = fem_tfidf_df.sum(axis=0)
tfidf_counts_fem.sort_values(ascending=False)[0:10]

wife ha             976.118614
men family          729.171984
fellow              677.549175
thing               612.694714
divorce             607.185712
partner violence    605.775949
lifestyle           550.282282
june                537.683578
voting              513.169382
greatest            450.885487
dtype: float64

In [24]:
# pulling top 10 words/phrases for men TFIDF with English stopwords
# interesting that feminist is one of the top 10 words for mensrights
tfidf_counts_men = men_tfidf_df.sum(axis=0)
tfidf_counts_men.sort_values(ascending=False)[0:10]

woman       868.976790
men         826.650364
wa          579.965131
just        491.954841
like        476.984193
don         462.072782
people      423.281936
gt          407.147621
right       403.265087
feminist    391.284568
dtype: float64

##### Comparing single words & bigrams

In [25]:
# finding top ngrams 1-2
top_100_fem_ngrams_stop = tfidf_counts_fem.sort_values(ascending=False)[0:100]
top_100_men_ngrams_stop = tfidf_counts_men.sort_values(ascending=False)[0:100]

In [26]:
# finding similar ngrams in both
fem_men_ngrams = set(top_100_men_ngrams_stop.index) & set(top_100_fem_ngrams_stop.index)

In [27]:
# not a lot of similarity in smaller ngrams
fem_men_ngrams

{'thing'}

There isn't a lot of similarity when comparing ngrams 1-2 between the two subs - AskFeminists has more common 2-word phrases (with stopwords removed) which suggest more similar contextualization of common words such as 'men' and 'women'. MensRights has very few 2-word phrases that are common, so the top 100 words/short phrases come with less context than the list for AskFeminists.

#### Analyzing top words (ngrams=1)

In [28]:
tfidf1 = TfidfVectorizer(max_features = 30000, stop_words = 'english')

In [29]:
# fitting to tfidf
fem_tfidf = tfidf1.fit_transform(fem['lems'])

In [30]:
fem_tfidf.shape

(28955, 30000)

In [31]:
men_tfidf = tfidf1.fit_transform(men['lems'])

In [32]:
# creating a df for tfidf words
fem_tfidf_df = pd.DataFrame(fem_tfidf.toarray(), columns=tfidf1.get_feature_names())

In [33]:
men_tfidf_df = pd.DataFrame(men_tfidf.toarray(), columns=tfidf1.get_feature_names())

In [34]:
# top words for askfeminists
tfidf_counts_fem = fem_tfidf_df.sum(axis=0)
tfidf_counts_fem.sort_values(ascending=False)[0:10]

workshop     1059.252157
manigault    803.501338 
fanedit      752.867745 
doesnt       665.165769 
tomboy       655.855623 
paypal       645.690254 
lawyered     582.257494 
jordon       572.694283 
weak         530.538017 
hardwick     474.213095 
dtype: float64

In [35]:
# top words for mensrights
tfidf_counts_men = men_tfidf_df.sum(axis=0)
tfidf_counts_men.sort_values(ascending=False)[0:10]

woman       932.960891
men         894.450155
wa          598.002861
just        512.151036
don         496.218393
like        493.284077
people      443.586868
right       435.860838
feminist    409.632239
gt          408.652878
dtype: float64

In [36]:
# finding top ngrams 1-2
top_100_fem_words_stop = tfidf_counts_fem.sort_values(ascending=False)[0:100]
top_100_men_words_stop = tfidf_counts_men.sort_values(ascending=False)[0:100]

In [37]:
# finding similar words in both
fem_men_words = set(top_100_men_words_stop.index) & set(top_100_fem_words_stop.index)

In [38]:
len(fem_men_words) # 77 of the words are in the top 100 for both! 

0

In [39]:
fem_men_words

set()

#### Comparing Stopwords with Count Vect

In [40]:
vect1 = CountVectorizer(max_features = 30000, stop_words = 'english')

In [41]:
# fitting to vect
fem_vect = vect1.fit_transform(fem['lems'])

In [42]:
men_vect = vect1.fit_transform(men['lems'])

In [43]:
# creating a df for tfidf words
fem_vect_df = pd.DataFrame(fem_vect.toarray(), columns=vect1.get_feature_names())

In [44]:
men_vect_df = pd.DataFrame(men_vect.toarray(), columns=vect1.get_feature_names())

In [45]:
# top words for askfeminists
vect_counts_fem = fem_vect_df.sum(axis=0)
vect_counts_fem.sort_values(ascending=False)[0:10]

workshop     22124
manigault    14360
paypal       12063
doesnt       11036
tomboy       10884
lawyered     10296
fanedit      10242
jordon       9659 
weak         9085 
hardwick     7853 
dtype: int64

In [46]:
# top words for mensrights
vect_counts_men = men_vect_df.sum(axis=0)
vect_counts_men.sort_values(ascending=False)[0:10]

woman       17622
men         16400
wa          10868
just        7402 
like        6929 
people      6385 
don         6367 
gt          6236 
right       5382 
feminist    5265 
dtype: int64

In [47]:
# finding top words
top_100_fem_words_stop = vect_counts_fem.sort_values(ascending=False)[0:100]
top_100_men_words_stop = vect_counts_men.sort_values(ascending=False)[0:100]

In [48]:
# finding similar words in both
fem_men_words = set(top_100_men_words_stop.index) & set(top_100_fem_words_stop.index)

In [49]:
len(fem_men_words) # about the same as tfidf

0

#### Setting Custom Stopwords, Second Comparison

In [50]:
# Brian's code, setting stop words equal to intersection terms in top 100 using count vectorizer
stop_words_1 = text.ENGLISH_STOP_WORDS.union(['actually',
 'agree',
 'bad',
 'believe',
 'better',
 'case',
 'child',
 'come',
 'comment',
 'did',
 'didn',
 'doe',
 'doesn',
 'don',
 'equality',
 'fact',
 'feel',
 'female',
 'feminism',
 'feminist',
 'gender',
 'girl',
 'going',
 'good',
 'group',
 'gt',
 'guy',
 'ha',
 'having',
 'help',
 'isn',
 'issue',
 'just',
 'know',
 'life',
 'like',
 'look',
 'lot',
 'make',
 'male',
 'man',
 'mean',
 'men',
 'need',
 'people',
 'person',
 'point',
 'post',
 'problem',
 'rape',
 'read',
 'really',
 'reason',
 'right',
 'said',
 'say',
 'saying',
 'sex',
 'sexual',
 'society',
 'sure',
 'thanks',
 'thing',
 'think',
 'thought',
 'time',
 'use',
 'victim',
 'wa',
 'want',
 'way',
 'white',
 'woman',
 'word',
 'work',
 'wrong',
 'yes'])

In [51]:
tfidf2 = TfidfVectorizer(max_features = 30000, stop_words = stop_words_1)

In [52]:
# fitting to tfidf
fem_tfidf = tfidf2.fit_transform(fem['lems'])

In [53]:
men_tfidf = tfidf2.fit_transform(men['lems'])

In [54]:
# creating a df for tfidf words
fem_tfidf_df = pd.DataFrame(fem_tfidf.toarray(), columns=tfidf2.get_feature_names())

In [55]:
men_tfidf_df = pd.DataFrame(men_tfidf.toarray(), columns=tfidf2.get_feature_names())

In [56]:
# top words for askfeminists, after removing most common words
tfidf_counts_fem = fem_tfidf_df.sum(axis=0)
tfidf_counts_fem.sort_values(ascending=False)[0:10]

relationshipi    378.489849
herald           223.135285
dialed           219.112282
unoriginal       204.333295
anatomist        204.207552
partyi           195.403570
eradicate        191.807080
trustworthy      182.818588
unbroken         178.130618
applies          177.842142
dtype: float64

In [57]:
# top words for mensrights, after removing most common words
tfidf_counts_men = men_tfidf_df.sum(axis=0)
tfidf_counts_men.sort_values(ascending=False)[0:10]

year       257.012902
day        203.796288
archive    202.565864
boy        193.050981
law        172.890177
sub        170.635181
article    166.782898
got        164.916142
shit       163.609884
support    161.837291
dtype: float64

In [58]:
# finding top words
top_100_fem_words_stop1 = tfidf_counts_fem.sort_values(ascending=False)[0:100]
top_100_men_words_stop1 = tfidf_counts_men.sort_values(ascending=False)[0:100]

In [59]:
# finding similar words in both
fem_men_words1 = set(top_100_men_words_stop1.index) & set(top_100_fem_words_stop1.index)

In [60]:
len(fem_men_words1)

0

In [61]:
fem_men_words1

set()

### Setting Custom Stopwords (2nd layer)

In [62]:
# added common words from last version to common words from previous version
stop_words_2 = text.ENGLISH_STOP_WORDS.union(['agree',
 'aren',
 'argument',
 'article',
 'assault',
 'best',
 'boy',
 'care',
 'change',
 'claim',
 'consent',
 'crime',
 'day',
 'different',
 'doing',
 'equal',
 'evidence',
 'exactly',
 'example',
 'far',
 'friend',
 'getting',
 'got',
 'hard',
 'hate',
 'idea',
 'job',
 'kind',
 'law',
 'let',
 'making',
 'masculinity',
 'matter',
 'maybe',
 'movement',
 'opinion',
 'place',
 'power',
 'pretty',
 'probably',
 'question',
 'read',
 'real',
 'seen',
 'sound',
 'stop',
 'study',
 'sub',
 'talk',
 'talking',
 'tell',
 'thanks',
 'toxic',
 'true',
 'try',
 'understand',
 'used',
 'violence',
 'word',
 'wouldn',
 'yeah',
 'yes',
'actually',
 'agree',
 'bad',
 'believe',
 'better',
 'case',
 'child',
 'come',
 'comment',
 'did',
 'didn',
 'doe',
 'doesn',
 'don',
 'equality',
 'fact',
 'feel',
 'female',
 'feminism',
 'feminist',
 'gender',
 'girl',
 'going',
 'good',
 'gt',
 'guy',
 'ha',
 'isn',
 'issue',
 'just',
 'know',
 'life',
 'like',
 'look',
 'lot',
 'make',
 'male',
 'man',
 'mean',
 'men',
 'nan',
 'need',
 'people',
 'person',
 'point',
 'post',
 'problem',
 'rape',
 'read',
 'really',
 'reason',
 'right',
 'said',
 'say',
 'saying',
 'sex',
 'sexual',
 'sure',
 'thanks',
 'thing',
 'think',
 'time',
 'use',
 'wa',
 'want',
 'way',
 'white',
 'woman',
 'word',
 'work',
 'wrong',
 'yes', 'aren',
 'argument',
 'assault',
 'best',
 'boy',
 'care',
 'change',
 'claim',
 'consent',
 'day',
 'different',
 'doing',
 'equal',
 'evidence',
 'exactly',
 'far',
 'getting',
 'got',
 'group',
 'hate',
 'having',
 'help',
 'idea',
 'job',
 'kind',
 'law',
 'le',
 'let',
 'making',
 'matter',
 'maybe',
 'movement',
 'ok',
 'place',
 'power',
 'pretty',
 'probably',
 'question',
 'real',
 'society',
 'sound',
 'stop',
 'sub',
 'support',
 'talk',
 'talking',
 'tell',
 'thought',
 'true',
 'try',
 'trying',
 'understand',
 'used',
 'victim',
 'violence',
 'world',
 'wouldn',
 'yeah',
 'year'])

### Combining into one DF

In [63]:
femvmen = pd.concat([fem, men], axis=0, join='outer')

In [64]:
femvmen.shape

(57955, 8)

In [65]:
femvmen.columns

Index(['Unnamed: 0', 'text', 'type', 'subreddit', 'removed', 'deleted',
       'clean_text_stop', 'lems'],
      dtype='object')

In [66]:
femvmen = pd.get_dummies(femvmen, columns=['subreddit'], drop_first = True)

In [67]:
femvmen.drop(columns = "Unnamed: 0", inplace = True)

In [68]:
femvmen.reset_index(inplace = True)

In [69]:
femvmen = pd.DataFrame(femvmen)

In [70]:
femvmen.to_csv('./data/femvmen_lem')

In [71]:
femvmen.columns

Index(['index', 'text', 'type', 'removed', 'deleted', 'clean_text_stop',
       'lems', 'subreddit_MensRights'],
      dtype='object')

# Modeling <a name="model"></a>

#### Setting Vars/TrainTestSplit

In [72]:
X = femvmen['lems']
y = femvmen['subreddit_MensRights']

In [73]:
y.value_counts(normalize = True) # about 50% mensrights posts

1    0.500388
0    0.499612
Name: subreddit_MensRights, dtype: float64

In [74]:
#TRAIN TEST SPLIT
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=True, 
                                                    stratify = y)

In [75]:
y_train.value_counts(normalize = True)

1    0.500388
0    0.499612
Name: subreddit_MensRights, dtype: float64

#### Vectorizing

In [76]:
# setting up multiple vectorizers with english stopwords, ngrams 1-4, different features
tfidf = TfidfVectorizer(ngram_range = (1, 4), max_features = 125000, stop_words = 
                        'english')
tfidf2 = TfidfVectorizer(ngram_range = (1, 4), max_features = 125000, stop_words = 
                        stop_words_1)
tfidf3 = TfidfVectorizer(ngram_range = (1, 4), max_features = 125000, stop_words = 
                        stop_words_2)
tfidf4 = TfidfVectorizer(ngram_range = (1, 4), max_features = 100000, stop_words = 
                        'english')
tfidf5 = TfidfVectorizer(ngram_range = (1, 4), max_features = 100000, stop_words = 
                        stop_words_1)
tfidf6 = TfidfVectorizer(ngram_range = (1, 4), max_features = 100000, stop_words = 
                        stop_words_2)
tfidf7 = TfidfVectorizer(ngram_range = (1, 4), max_features = 50000, stop_words = 
                        'english')
tfidf8 = TfidfVectorizer(ngram_range = (1, 4), max_features = 50000, stop_words = 
                        stop_words_1)
tfidf9 = TfidfVectorizer(ngram_range = (1, 4), max_features = 50000, stop_words = 
                        stop_words_2)
tfidf10 = TfidfVectorizer(ngram_range = (1, 4), max_features = 5000, stop_words = 
                        'english')
tfidf11 = TfidfVectorizer(ngram_range = (1, 4), max_features = 5000, stop_words = 
                        stop_words_1)
tfidf12 = TfidfVectorizer(ngram_range = (1, 4), max_features = 5000, stop_words = 
                        stop_words_2)

In [77]:
#fit transform train sets
X_train_tf = tfidf.fit_transform(X_train)
X_train_tf2 = tfidf2.fit_transform(X_train)
X_train_tf3 = tfidf3.fit_transform(X_train)
X_train_tf4 = tfidf4.fit_transform(X_train)
X_train_tf5 = tfidf5.fit_transform(X_train)
X_train_tf6 = tfidf6.fit_transform(X_train)
X_train_tf7 = tfidf7.fit_transform(X_train)
X_train_tf8 = tfidf8.fit_transform(X_train)
X_train_tf9 = tfidf9.fit_transform(X_train)
X_train_tf10 = tfidf10.fit_transform(X_train)
X_train_tf11 = tfidf11.fit_transform(X_train)
X_train_tf12 = tfidf12.fit_transform(X_train)

In [78]:
# transform test set
X_test_tf = tfidf.transform(X_test)
X_test_tf2 = tfidf2.transform(X_test)
X_test_tf3 = tfidf3.transform(X_test)
X_test_tf4 = tfidf4.transform(X_test)
X_test_tf5 = tfidf5.transform(X_test)
X_test_tf6 = tfidf6.transform(X_test)
X_test_tf7 = tfidf7.transform(X_test)
X_test_tf8 = tfidf8.transform(X_test)
X_test_tf9 = tfidf9.transform(X_test)
X_test_tf10 = tfidf10.transform(X_test)
X_test_tf11 = tfidf11.transform(X_test)
X_test_tf12 = tfidf12.transform(X_test)

### Logistic Regression

Best versions of vectorized data for logistic regression are with stopwords = 'english' and max features = 125000. With very few features (10K or less) the gap between train and test scores closes to 3% but the test score continues to drop with fewer than 100K features.

In [79]:
logreg = LogisticRegression(penalty='l2')

In [80]:
# version 1
X_train_tf = tfidf.fit_transform(X_train)

In [81]:
X_test_tf = tfidf.transform(X_test)

In [82]:
logreg.fit(X_train_tf, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [83]:
logreg.score(X_train_tf, y_train)

0.853420757484255

In [84]:
logreg.score(X_test_tf, y_test)

0.7560175998619618

In [85]:
# version 2
logreg.fit(X_train_tf2, y_train)
logreg.score(X_train_tf2, y_train)

0.8392718488482444

In [86]:
logreg.score(X_test_tf2, y_test)

0.7379863687343629

In [87]:
# version 3
logreg.fit(X_train_tf3, y_train)
logreg.score(X_train_tf3, y_train)

0.8339228711931671

In [88]:
logreg.score(X_test_tf3, y_test)

0.7366059874040204

In [89]:
# version 4
logreg.fit(X_train_tf4, y_train)
logreg.score(X_train_tf4, y_train)

0.8494737296178069

In [90]:
logreg.score(X_test_tf4, y_test)

0.7559313260288154

In [91]:
# version 5
logreg.fit(X_train_tf5, y_train)
logreg.score(X_train_tf5, y_train)

0.8349365887326374

In [92]:
logreg.score(X_test_tf5, y_test)

0.739711845397291

In [93]:
# version 6
logreg.fit(X_train_tf6, y_train)
logreg.score(X_train_tf6, y_train)

0.829868001035286

In [94]:
logreg.score(X_test_tf6, y_test)

0.7355707014062635

In [95]:
# version 7
logreg.fit(X_train_tf7, y_train)
logreg.score(X_train_tf7, y_train)

0.8363385385212665

In [96]:
logreg.score(X_test_tf7, y_test)

0.7547234923647658

In [97]:
# version 8
logreg.fit(X_train_tf8, y_train)
logreg.score(X_train_tf8, y_train)

0.8239582434647571

In [98]:
logreg.score(X_test_tf8, y_test)

0.739280476231559

In [99]:
# version 9
logreg.fit(X_train_tf9, y_train)
logreg.score(X_train_tf9, y_train)

0.8189974980588387

In [100]:
logreg.score(X_test_tf9, y_test)

0.736519713570874

In [101]:
# version 10
logreg.fit(X_train_tf10, y_train)
logreg.score(X_train_tf10, y_train)

0.7811664222241395

In [102]:
logreg.score(X_test_tf10, y_test)

0.7394530238978518

In [103]:
# version 11
logreg.fit(X_train_tf11, y_train)
logreg.score(X_train_tf11, y_train)

0.7710292468294366

In [104]:
logreg.score(X_test_tf11, y_test)

0.7278923302562332

In [105]:
# version 12
logreg.fit(X_train_tf12, y_train)
logreg.score(X_train_tf12, y_train)

0.7657449745492192

In [106]:
logreg.score(X_test_tf12, y_test)

0.7192649469415926

### Most Influential Features

In [107]:
# fitting on best model
logreg.fit(X_train_tf, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [108]:
coefficients = logreg.coef_

In [109]:
coef_df = pd.DataFrame(coefficients, columns = tfidf2.get_feature_names())

In [110]:
coef_df.head()

,aa,aaaaand,aap,aap circumcision,aauw,ab,aback,abandon,abandoned,abandoning,abandonment,abbey,abbreviated,abbreviation,abc,abc news,abctv,abctv scottmorrisonmp,abd,abdicate,abdicate responsibility,abdication,abdomen,abdominal,abducted,abduction,abelist,aberrant,aberration,abhor,abhor culture,abhorrent,abide,abiding,abiding citizen,abigail,ability,ability based,ability birth,ability change,ability choose,ability commit,ability compete,ability consent,ability consent reasonable,ability control,ability create,ability decision,ability dress,ability dress certain,ability earn,ability enjoy,ability experience,ability influence,ability informed,ability job,ability le,ability obtain,ability pay,ability power,ability prevent,ability provide,ability provide care,ability refrain,ability refrain personal,ability refrain personal accountability,ability reproduce,ability responsibility,ability survive,ability understand,ability willingness,abit,abject,able,able abandon,able able,able abortion,able access,able accrue,able accrue wealth,able achieve,able afford,able angry,able answer,able answer question,able argue,able argument,able ask,able away,able bodied,able bodied ci,able bring,able buy,able care,able change,able choose,able claim,able compete,able consent,able consider,able continue,able contribute,able control,able convince,able create,able date,able day,able deal,able decide,able decision,able defend,able discriminate,able discus,able distinguish,able empathize,able evidence,able experience,able explain,able explain biological,able explain biological basis,able express,able fight,able financial,able fix,able force,able forward,able fully,able gain,able handle,able hold,able identify,able incorporate,able job,able learn,able leave,able legally,able lift,able live,able maintain,able manipulate,able meet,able navigate,able opt,able participate,able perform,able pick,able play,able pregnant,able protect,able prove,able provide,able pull,able raise,able recognize,able record,able refute,able relate,able remove,able separate,able separate conversation,able separate conversation fgm,able set,able sexist,able share,able shut,able sign,able solve,able sort,able speak,able speak clearly,able speak clearly unambiguously,able spend,able stand,able start,able stay,able study,able support,able survive,able talk,able tell,able train,able trust,able turn,able understand,able vote,able watch,able wear,abled,ableism,ableist,ableist slur,abnormal,abnormal behavior,abnormality,aboard,abolish,abolish prison,abolished,abolishing,abolition,abolition capitalism,abolitionism,abolitionist,abominable,abomination,aboriginal,abort,abort baby,abort body,abort fetus,abort force,aborted,aborted fetus,aborting,aborting baby,abortion,abortion able,abortion abortion,abortion access,abortion adoption,abortion aren,abortion argument,abortion available,abortion baby,abortion based,abortion bc,abortion begin,abortion birth,abortion birth control,abortion bodily,abortion bodily autonomy,abortion body,abortion body choice,abortion choice,abortion clinic,abortion completely,abortion consent,abortion consider,abortion cost,abortion country,abortion debate,abortion easy,abortion effective,abortion fine,abortion form,abortion free,abortion freely,abortion gendered,abortion generally,abortion happening,abortion illegal,abortion killing,abortion late,abortion law,abortion le,abortion legal,abortion let,abortion literally,abortion making,abortion medical,abortion month,abortion mother,abortion murder,abortion necessarily,abortion okay,abortion option,abortion parent,abortion pay,abortion poor,abortion position,abortion pregnancy,abortion pregnant,abortion pretty,abortion prevent,abortion pro,abortion pro choice,abortion rare,abortion reproductive,abortion service,abortion shouldn,abortion simply,abortion support,abortion topic,abortion unjust,abortion unjust killing,abortion wanted,abortion week,abortion world,abovementioned,abraham,abrahamic,abrahamic religion,abrams,abrasion,ab

In [111]:
# words most positively correlated with content being from MensRights
# some of these make NO sense, might have been good to do more cleaning
coef_df.sum(axis=0).sort_values(ascending=False)[0:15]

military base                    5.529991
broad generalization             4.476077
article quote                    4.112461
shouldn question idea            3.429160
bewildered                       3.273144
minute slightly increased        3.245495
bro jap                          3.156064
masculinity boy                  3.101634
mra incel                        3.046819
shared common                    2.923378
extensively                      2.895841
experiential authority member    2.846495
mother choose                    2.794948
semantic                         2.685628
hear trans                       2.653834
dtype: float64

In [112]:
# words most positively correlated with content being from AskFeminists
# these make a little more sense than the MensRights ones...slightly.
coef_df.sum(axis=0).sort_values(ascending=True)[0:15]

pretty important law             -5.759800
tell laugh joke pretty           -4.878412
support accused                  -4.871705
end jail                         -3.916339
designated disposal area         -3.349880
open feeling                     -3.347517
single conversation              -3.258743
argument opinion                 -3.215864
haram                            -3.131597
dominated dominated              -3.064219
argument waste                   -3.058328
fair deserve miserable death     -3.045156
physical violent abuse sharing   -3.019294
muslim absolutely equal          -2.755272
britain agonizing day            -2.743639
dtype: float64

### Confusion Matrix

In [113]:
pred_log = logreg.predict(X_test_tf)

In [114]:
pred_log.shape

(11591,)

In [115]:
y_test.shape

(11591,)

In [116]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred_log)
tn, fp, fn, tp = confusion_matrix(y_test, pred_log).ravel()
print("True Negatives: %s" % tn)
print("False Positives: %s" % fp)
print("False Negatives: %s" % fn)
print("True Positives: %s" % tp)

True Negatives: 4266
False Positives: 1525
False Negatives: 1303
True Positives: 4497


In [117]:
# setting up df for comparison
y_test_preds = pd.DataFrame(y_test)

In [118]:
y_test_preds['pred'] = pred_log

In [119]:
y_test_preds.head()

,subreddit_MensRights,pred
5938,0,0
153,0,0
7102,0,0
20531,0,1
32532,1,1


In [120]:
# rows where predictions don't match
y_test_preds[y_test_preds['subreddit_MensRights'] != y_test_preds['pred']].head()

,subreddit_MensRights,pred
20531,0,1
48911,1,0
27098,0,1
17941,0,1
29097,1,0


In [121]:
# example of a post misclassified as AskFeminists
#this comment is just responding to another user who posted something about JP (Jordan Peterson)
# but there's no real context that would help identify which thread this belongs to
femvmen.loc[57021]

index                   19820                                                                                                                                                                                                                                                   
text                    The first clip legitimately doesn't work, and none from the second are JP so I've still not seen proof of your origional claim that JP ever said that. If you send me a functioning clip then I'll change my opinion but it just says video unavailable.
type                    comment                                                                                                                                                                                                                                                 
removed                 0                                                                                                                                                            

In [122]:
# example of a post misclassified as MensRights
# not much to go off here for classification, it could really be from either.
femvmen.loc[18964]

index                   19170                                                                     
text                    Dude, I can't argue with you when you are so deep in the dogma. Good luck.
type                    comment                                                                   
removed                 0                                                                         
deleted                 0                                                                         
clean_text_stop         dude  i can t argue with you when you are so deep in the dogma  good luck 
lems                    dude i can argue with you when you are so deep in the dogma good luck     
subreddit_MensRights    0                                                                         
Name: 18964, dtype: object

### Other models

I tried all of the below models before selecting the Logistic Regression for further analysis of the coefficients - none of these had as strong a test score as the one on the Logistic Regression. I spent more time with Random Forest, and I could have tried to tune the parameters for Naive Bayes and SVC to improve the scores but didn't have time.

### Naive Bayes Multinomial

In [123]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [124]:
nb.fit(X_train_tf2, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [125]:
nb.score(X_train_tf2, y_train)

0.8530109567768096

In [126]:
nb.score(X_test_tf2, y_test)

0.7464412043827108

## Support Vector Classifier

In [127]:
from sklearn import svm
svc = svm.SVC(kernel= 'rbf', C = 100, gamma = 0.05)

In [128]:
svc.fit(X_train_tf, y_train)

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.05, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [129]:
svc.score(X_train_tf, y_train)

0.9867785350703132

In [130]:
svc.score(X_test_tf, y_test)

0.7386765593995341

## Random Forest

In [131]:
from sklearn.ensemble import RandomForestClassifier

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [132]:
# taking sample for faster gridsearching
sample20000 = femvmen.sample(n=20000, random_state = 42)

In [133]:
sample20000['subreddit_MensRights'].value_counts(normalize=True) # about same proportions as full dataset

1    0.5018
0    0.4982
Name: subreddit_MensRights, dtype: float64

In [134]:
X = sample20000['lems']
y = sample20000['subreddit_MensRights']

In [135]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size = .30, 
                                                   stratify=y)

In [136]:
tfidf = TfidfVectorizer(ngram_range = (1, 4), max_features = 125000, stop_words = 
                        'english')

In [137]:
X_train_tf = tfidf.fit_transform(X_train)
X_test_tf = tfidf.transform(X_test)

In [138]:
X_train_tf.shape

(14000, 125000)

In [139]:
# just trying out basic random forest without tuning parameters or setting max depth
rf = RandomForestClassifier()

In [140]:
rf.fit(X_train_tf, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [141]:
rf.score(X_train_tf, y_train)

0.981

In [142]:
rf.score(X_test_tf, y_test)

0.6566666666666666

In [143]:
rf = RandomForestClassifier()

In [144]:
# gridsearching for best parameters - this is the first one I did
from sklearn.model_selection import GridSearchCV

In [145]:
params = {
    'n_estimators': [5, 10, 1],
    'max_depth': [10000, 90000, 20000],
    'oob_score': ['True', 'False'],
    'warm_start': ['True'],
    'n_jobs': [-2]
}
gs = GridSearchCV(rf, param_grid = params, cv = 3)
gs.fit(X_train_tf, y_train)
print(gs.best_score_)
gs.best_params_ 

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees we

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees we

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees we

0.6554285714285715


/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


{'max_depth': 90000,
 'n_estimators': 10,
 'n_jobs': -2,
 'oob_score': 'False',
 'warm_start': 'True'}

In [146]:
# second gridsearch, I repeated this until I narrowed down the best max depth range
params = {
    'n_estimators': [10],
    'max_depth': range(875, 900, 1),
    'oob_score': ['False'],
    'warm_start': ['True'],
    'n_jobs': [-2]
}
gs = GridSearchCV(rf, param_grid = params, cv = 3)
gs.fit(X_train_tf, y_train)
print(gs.best_score_)
gs.best_params_ 

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees we

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees we

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees we

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees we

0.6618571428571428


/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


{'max_depth': 885,
 'n_estimators': 10,
 'n_jobs': -2,
 'oob_score': 'False',
 'warm_start': 'True'}

In [147]:
# resetting X and y variables to test random forest on full train/test sets
X = femvmen['lems']
y = femvmen['subreddit_MensRights']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = .25,
                                                   stratify = y)

In [148]:
X_train_tf = tfidf.fit_transform(X_train)
X_test_tf = tfidf.transform(X_test)

In [149]:
rf = RandomForestClassifier(max_depth = 880, n_estimators = 1000, n_jobs = -2, oob_score = False,
                 warm_start = True)

In [150]:
rf.fit(X_train_tf, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=880, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-2,
            oob_score=False, random_state=None, verbose=0, warm_start=True)

In [151]:
rf.score(X_train_tf, y_train)

0.984401601251553

In [152]:
rf.score(X_test_tf, y_test)

0.7180619780523155

## Conclusion

It would be too easy to say that my best model's inability to crack 77% accuracy means that "we're really not that different after all", and if I were to spend more time on this project there are other options I would explore to try and improve my model:

1. I'd pull a completely unrelated subreddit to test as a control against the other two subreddits to make sure I was tuning the best model possible.

2. Given how important context is for the comments, I would either try to aggregate comments on a post and analyze them together, or set a minimum wordcount on comments to be used in the analysis to give the model a better chance of distinguishing them.

3. I could scrape another years' worth of data to add to the model.

4. I could do some more cleaning and EDA that might help consolidate slang/similar terms that weren't captured by the lemmatizer, and identify stronger trends in the subreddits that could be leveraged for better prediction. It'd also be interesting to see what difference it might make to fit the TFIDF on just one subreddit first, then transform both subreddits and analyze them together.

5. If any of the above helped minimize overfit on the Logistic Regression or Random Forest, I'd give Naive Bayes Multinomial and Support Vector Classifier another shot (and spend more time tuning them).

The really interesting questions this project has generated would require a deeper analysis: what's the overlap in people who post on MensRights and AskFeminists? (How many of those are trolls, whose content is removed?) What common themes exist between removed posts (can we write an algorithm for trolling content?) How does sentiment analysis compare across the two subreddits, is there a discernable difference? Is it possible to measure 'extreme' attitudes in a subreddit, and if so can it be mapped over time, against real events happening in the world that might trigger anger on both or one side? Is it possible to follow the posts of single users and see how they change in sentiment and neutrality over time (and does the sub they post on the most make a difference?) Maybe I'll come back to it when I've solidified my modeling skills.